# Skip to the bottom for the netowrkx code

In [ ]:
import warnings

import geopandas
import libpysal
import momepy
import pandas
import geopy
import numpy


from clustergram import Clustergram

import matplotlib.pyplot as plt
from bokeh.io import output_notebook
from bokeh.plotting import show
from shapely.geometry import Point
import matplotlib.colors as mcolors

import osmnx

In [ ]:
local_crs = 27700
latlng = (55.86, -4.25)
dist = 5000

print("test")

In [ ]:
# buildings = osmnx.geometries.geometries_from_place(place, tags={'building':True})
# buildings = osmnx.geometries.geometries_from_place(place, tags={'building':True})
buildings = osmnx.geometries.geometries_from_point(latlng, tags={'building':True}, dist=dist)
buildings.head()

In [ ]:
buildings.head()

print(buildings.geom_type.value_counts())
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)
buildings = buildings[["geometry"]].to_crs(local_crs)

Above loads building into a geojson file

In [ ]:
buildings["uID"] = range(len(buildings))
buildings.head()

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
buildings.plot(ax=ax)
ax.set_axis_off()
plt.show()

In [ ]:
osm_graph = osmnx.graph_from_point(latlng, dist=dist, network_type='drive')
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)
streets = osmnx.graph_to_gdfs(
    osm_graph,
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
)
streets.head()

In [ ]:
streets = momepy.remove_false_nodes(streets)
streets = streets[["geometry"]]
streets["nID"] = range(len(streets))


In [ ]:
limit = momepy.buffered_limit(buildings, 100)

tessellation = momepy.Tessellation(buildings, "uID", limit, verbose=True, segment=1)
tessellation = tessellation.tessellation

In [ ]:
buildings = buildings.sjoin_nearest(streets, max_distance=1000, how="left")
buildings.head()

In [ ]:
buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [ ]:
f, ax = plt.subplots(figsize=(100, 100))
tessellation.plot(ax=ax, edgecolor='black')
buildings.plot(ax=ax, color='white', alpha=.5)
plt.show()

In [ ]:
buildings["area"] = buildings.area
tessellation["area"] = tessellation.area
streets["length"] = streets.length

buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series
buildings['elongation'] = momepy.Elongation(buildings).series
tessellation['convexity'] = momepy.Convexity(tessellation).series
streets["linearity"] = momepy.Linearity(streets).series

fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("eri", ax=ax[0], scheme="natural_breaks", legend=True)
buildings.plot("elongation", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

tessellation.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
streets.plot("linearity", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
buildings["shared_walls"] = momepy.SharedWallsRatio(buildings).series
buildings.plot("shared_walls", figsize=(12, 12), scheme="natural_breaks", legend=True).set_axis_off()

In [ ]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)

In [ ]:
tessellation.geom_type.value_counts()

tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series
tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_1, "uID", verbose=False).series

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    buildings["neighbor_distance"] = momepy.NeighborDistance(buildings, queen_1, "uID", verbose=False).series

fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("neighbor_distance", ax=ax[0], scheme="natural_breaks", legend=True)
tessellation.plot("covered_area", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
queen_3 = momepy.sw_high(k=3, weights=queen_1)
buildings_q1 = libpysal.weights.contiguity.Queen.from_dataframe(buildings, silence_warnings=True)

buildings['interbuilding_distance'] = momepy.MeanInterbuildingDistance(buildings, queen_1, 'uID', queen_3, verbose=False).series
buildings['adjacency'] = momepy.BuildingAdjacency(buildings, queen_3, 'uID', buildings_q1, verbose=False).series

fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("interbuilding_distance", ax=ax[0], scheme="natural_breaks", legend=True)
buildings.plot("adjacency", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
profile = momepy.StreetProfile(streets, buildings)
streets["width"] = profile.w
streets["width_deviation"] = profile.wd
streets["openness"] = profile.o

fig, ax = plt.subplots(1, 3, figsize=(24, 12))

streets.plot("width", ax=ax[0], scheme="natural_breaks", legend=True)
streets.plot("width_deviation", ax=ax[1], scheme="natural_breaks", legend=True)
streets.plot("openness", ax=ax[2], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

In [ ]:
tessellation['car'] = momepy.AreaRatio(tessellation, buildings, 'area', 'area', 'uID').series
tessellation.plot("car", figsize=(12, 12), vmin=0, vmax=1, legend=True).set_axis_off()

In [ ]:
graph = momepy.gdf_to_nx(streets)
graph = momepy.node_degree(graph)
graph = momepy.closeness_centrality(graph, radius=400, distance="mm_len")
graph = momepy.meshedness(graph, radius=400, distance="mm_len")
nodes, streets = momepy.nx_to_gdf(graph)


fig, ax = plt.subplots(1, 3, figsize=(24, 12))

nodes.plot("degree", ax=ax[0], scheme="natural_breaks", legend=True, markersize=1)
nodes.plot("closeness", ax=ax[1], scheme="natural_breaks", legend=True, markersize=1, legend_kwds={"fmt": "{:.6f}"})
nodes.plot("meshedness", ax=ax[2], scheme="natural_breaks", legend=True, markersize=1)

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

In [ ]:
buildings["nodeID"] = momepy.get_node_id(buildings, nodes, streets, "nodeID", "nID")

In [ ]:
tessellation.head()

In [ ]:
merged = tessellation.merge(buildings.drop(columns=['nID', 'geometry']), on='uID')
merged = merged.merge(streets.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')

In [ ]:
merged.columns

In [ ]:
percentiles = []
for column in merged.columns.drop(["uID", "nodeID", "nID", 'mm_len', 'node_start', 'node_end', "geometry"]):
    perc = momepy.Percentiles(merged, column, queen_3, "uID", verbose=False).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)

percentiles_joined = pandas.concat(percentiles, axis=1)
percentiles_joined.head()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

tessellation.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
merged.plot(percentiles_joined['convexity_50'].values, ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
standardized = (percentiles_joined - percentiles_joined.mean()) / percentiles_joined.std()
standardized.head()

In [ ]:
n=12

cgram = Clustergram(range(1, n), n_init=10, random_state=42)
cgram.fit(standardized.fillna(0))

show(cgram.bokeh())

In [ ]:
cgram.labels.head()

In [ ]:
merged["cluster"] = cgram.labels[n-1].values
urban_types = buildings[["geometry", "uID"]].merge(merged[["uID", "cluster"]], on="uID")

In [ ]:
# Color


pastel1_cmap = plt.get_cmap('Pastel1')
values = list(set(cgram.labels[n-1].values))

color_lookup = {k: pastel1_cmap(k) for k in values}

cmap = mcolors.ListedColormap([color_lookup[val] for val in sorted(color_lookup)])


In [ ]:

urban_types.plot("cluster", categorical=True, figsize=(16, 16), legend=True, cmap=cmap).set_axis_off()

In [ ]:
urban_types_area = tessellation[["geometry", "uID"]].merge(merged[["uID", "cluster"]], on="uID")
urban_types_area.plot("cluster", categorical=True, figsize=(16, 16), legend=True, cmap=pastel1_cmap).set_axis_off()

In [ ]:
dissolved = urban_types_area.dissolve(by='cluster', aggfunc='count', as_index=False).explode(inex_parts=False)

min_area = 5000 # minimum area in square meters

# Calculate the area of each polygon in square meters
dissolved['area_m2'] = dissolved['geometry'].area

# Filter the GeoDataFrame to keep only polygons with an area greater than or equal to min_area
filtered = dissolved[dissolved['area_m2'] >= min_area]

# Drop the area column since it is no longer needed
filtered = filtered.drop(columns=['area_m2'])

filtered['uID'] = numpy.arange(len(filtered.index))

# assuming gdf is a GeoDataFrame
centroid_series = filtered.centroid

# assuming gdf is a GeoDataFrame
centroid_series = centroid_series.centroid

# extract the x and y coordinates of each centroid point
x = centroid_series.x
y = centroid_series.y

# compute the mean x and y coordinates separately
mean_x = numpy.mean(x)
mean_y = numpy.mean(y)

# create a new Point object for the mean centroid
mean_centroid = Point(mean_x, mean_y)

# compute the standard deviation of the centroid coordinates
centroid_coords = numpy.column_stack((centroid_series.x, centroid_series.y))
centroid_coords_std = centroid_coords.std(axis=0)
# normalize the centroid coordinates
centroid_coords_norm = (centroid_coords - mean_centroid.coords) / centroid_coords_std
centroid_coords_norm = [Point(x, y) for x, y in centroid_coords_norm]

filtered["centroid_norm"] = centroid_coords_norm

filtered.head()

In [ ]:
dissolved.to_file("urbantypes.shp")

In [ ]:
f, ax = plt.subplots(figsize=(100, 100))
filtered.plot(ax=ax, column="cluster", categorical=True, legend=True, cmap=cmap)
streets.plot(ax=ax, color='black')
buildings.plot(ax=ax, color='grey')
ax.set_axis_off()

# Network building

- The following code uses the data from above to construct a network of touching urban types.
- This is a preliminary test for a possible application in my dissertation
- Full disclosure - quite a bit of the code was written with the help of ChatGPT; it was especially helpful in pointing me to what functions to use in networkX whilst learning it.

In [ ]:
## module import

import networkx as nx
from itertools import combinations

In [ ]:
# Init graph object
G = nx.Graph()

In [ ]:
uID_list = filtered["uID"].unique()

for uid in numpy.sort(uID_list):
    G.add_node(uid)

# Check for adjacency and add it to the graph
for x, y in combinations(uID_list, 2):
    x_df = filtered[filtered["uID"] == x]
    y_df = filtered[filtered["uID"] == y]
    x_geom = x_df.iloc[0]['geometry']
    y_geom = y_df.iloc[0]['geometry']
    if x_geom.touches(y_geom):
        print(x, "touches", y)
        G.add_edge(x, y)
    elif x_geom.intersects(y_geom):
        print(x, "intersects", y)
        G.add_edge(x, y)

In [ ]:
# Position each node at the position of the centroid of each polygon, normalised for scaling.
attrs = {}
pos = {}

area = filtered.area

# define a normalization function
area_mean = numpy.mean(area)
area_std = numpy.std(area)

for index, row in filtered.iterrows():

    attrs[row["uID"]] = {
        "urban_type": row["cluster"],
        "area": area[index]
    }
    pos[row["uID"]] = (row["centroid_norm"].x, row["centroid_norm"].y)  

nx.set_node_attributes(G, attrs)

In [ ]:
# First rendering of graph
options = {
    "font_size": 0,
    "node_size": [((attr['area'] *12.5 - area_mean)/area_std) for (node, attr) in G.nodes(data=True)],
    "edgecolors": "black",
    "node_color": [color_lookup[attr['urban_type']] for (node, attr) in G.nodes(data=True)],
    "linewidths": 0.5,
    "width": 0.5,
}
nx.draw_networkx(G, pos, **options)

# Map for comparison
f, ax = plt.subplots(figsize=(100, 100))
filtered.plot(ax=ax, column="cluster", categorical=True, legend=True, cmap=cmap)
streets.plot(ax=ax, color='black')
buildings.plot(ax=ax, color='grey')
ax.set_axis_off()

# Comments

- The nodes in the graph represent each neighbourhood of homgenous urban type. The size of the node is a normalised value of the area of each of the neighbourhood, whilst the vertices represent the neighbourhoods that border each other. The nodes are positioned at a normalised value of their centoids. This graph is a representation of Glasgow
- We can observe from the graph that central Glasgow, streching from Partick to merchant city is realtively homogenous in urban form; They are mostly characterised by dual use rowhouses that are constrained by the gridded streets to be rectangular-ish, oriented roughly in the same direction, and having relatively similar "front yard-tage"; the streets themselves, as grids, presumably weighed heavily in the morphology
- High cliqueness would in this context will indicate small and highly variegated urban types within a small area; this is, curiously, the case for the East End around Camlachie.
- The gridded downtown is differentiated from the more peripherial parts of downtown Glasgow across the river and in the West end primarily due to differences in road network; the block size in the peripherial areas are larger, and have interior courtyards
- The largest neighbourhoods were orange. They are low density neighbourhoods where buildings are generally large, oddpped, and larger as a function of footprint. They form a belt around Glasgow's central city, snaking along important trunk roads. Most of these area has significant greenery - Glasgow Green, the Necropolis, Sighthill Cemetery, Cowlairs Park, Gartnavel hospital. These areas are largely contiguous and green, therefore becoming classified as a homogemous type in momepy with the largest size. I suspect this has to do with planning rules, but it was cool seeing it reflected in the data
- Their size and place as a barrier between the inner city and suburbs also means that these are generally the highest degree centrality; which is curious, because these green spaces are by intuition as a person living in Glasgow the most accessible Green spaces; and it is connected to a wide diversity of different types of buildings, perhaps indicating the diversity of people living in proximity to these spaces? More evidence needed to prove this correlation.
- "Single family houses", neighbourhoods with discrete family home-sized building werelly located in the suburbs - represented by the green, purple, and red urban. They show up effectively as regions whose houses were built by the same developer all at once.

In [ ]:
# Save netowrk to graph

import json
from networkx.readwrite import json_graph

# assuming you already have a NetworkX graph object called "G"
data = json_graph.node_link_data(G)

# convert int64 to int
for node in data['nodes']:
    node['id'] = int(node['id'])
for link in data['links']:
    link['source'] = int(link['source'])
    link['target'] = int(link['target'])

with open("network.json", "w") as outfile:
    json.dump(data, outfile)